### Import General Python libs, defined classes, and modules¶

In [1]:
from pprint import pprint
import mlflow
from mlflow.tracking import MlflowClient
from train_model import DriverRankingTrainModel
from predict_model import DriverRankingPredictModel

### Train the model

 * Use the `DriverRankingTrainingModel` class, which fetches training data from the Feast data source
 * Use `mlflow.autolog` to automatically log the parameters and computed metrics during training
 * Use `mflow` Fluent APIs to log the Feast training set

### Define ElasticNet tuning parameters and run some experiments

In [2]:
params_list = [{"alpha": 0.5, "l1_ratio": 0.15},
               {"alpha": 0.75, "l1_ratio": 0.25},
               {"alpha": 1.0, "l1_ratio": 0.5}]

# Change this to your location for the Feast feature repo
#REPO_PATH = "/Users/jsd/git-repos/feast_workshops/module_1/feature_repo"
REPO_PATH = "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-master/module_1/feature_repo"
# define your feature service name
FEATURE_SERVICE_NAME = "driver_ranking_fv_svc"

### Iterate and experiment MLflow runs tunning parameters

In [3]:
# iterate over tuning parameters
for params in params_list:
    model_cls = DriverRankingTrainModel(REPO_PATH, FEATURE_SERVICE_NAME, params)
    run_id = model_cls.train_model()
    pprint(f"ElasticNet params: {params}")
    print(f"Model run id: {run_id}")

2022/06/14 18:08:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kike/opt/miniconda3/envs/feast_workshop/lib/python3.10/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2022/06/14 18:08:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kike/opt/m

"ElasticNet params: {'alpha': 0.5, 'l1_ratio': 0.15}"
Model run id: {'d33f128b4b9a449a846755cb6b50285f'}


2022/06/14 18:08:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kike/opt/miniconda3/envs/feast_workshop/lib/python3.10/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


"ElasticNet params: {'alpha': 0.75, 'l1_ratio': 0.25}"
Model run id: {'b99ec692556b4cc5a52a89906b568437'}


2022/06/14 18:08:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kike/opt/miniconda3/envs/feast_workshop/lib/python3.10/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


"ElasticNet params: {'alpha': 1.0, 'l1_ratio': 0.5}"
Model run id: {'6a76cbad95424c3b9cb09e66f4dcf9e1'}


### Launch the MLflow ui, with Model Registry at the local SQLite database
 * Navigate and examine runs for the model
 * Register the best model with lowest RMSE with the Model Registry

In [4]:
#!mlflow ui  --backend-store-uri sqlite:///mlruns.db

### CI/CD Intergration: Fetch the registered model from the Model Registry
 * Use the model URI (either by stage or version)
 * Make the predicion

In [7]:
mlflow.set_tracking_uri("sqlite:///mlruns.db")
#REPO_PATH = "/Users/jsd/git-repos/feast_workshops/module_1/feature_repo"
REPO_PATH = "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-master/module_1/feature_repo"
#model_uri = "models:/sklearn_feast_integration/staging"
model_uri = "./mlruns/0/6a76cbad95424c3b9cb09e66f4dcf9e1/artifacts/model"
model = DriverRankingPredictModel(REPO_PATH, model_uri, FEATURE_SERVICE_NAME)
drivers = [1001, 1002, 1003]
best_driver = model(drivers)
print(f" Best predicted driver for completed trips: {best_driver}")

 Best predicted driver for completed trips: 1001


### CI/CD Integration: Transition the model to production 

In [8]:
client = MlflowClient()
client.transition_model_version_stage(
    name="sklearn_feast_integration",
    version=1,
    stage="Production"
)

<ModelVersion: creation_timestamp=1655248244875, current_stage='Production', description='', last_updated_timestamp=1655248287213, name='sklearn_feast_integration', run_id='6a76cbad95424c3b9cb09e66f4dcf9e1', run_link='', source='./mlruns/0/6a76cbad95424c3b9cb09e66f4dcf9e1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>